<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/Scrape_BUW_catalogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install scrapy
!pip3 install bs4

     |████████████████████████████████| 245kB 2.7MB/s 
     |████████████████████████████████| 2.7MB 8.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 3.2MB 26.5MB/s 
     |████████████████████████████████| 3.1MB 35.1MB/s 
     |████████████████████████████████| 235kB 37.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=7d08a09e95f7acdc32a8bc16067a15a4138aecab8b8f2a1d191a0eba05126660
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=df72f1a2a07c52135833d5a0934cbfafba3c8cbb8e974693c63e025613415e6b
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher


In [2]:
import scrapy
from bs4 import BeautifulSoup
import json
import re


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('Library_catalogue.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


class BookSpider(scrapy.Spider):
    name = 'books'
    start_urls = ['https://chamo.buw.uw.edu.pl/search/query?match_1=PHRASE&field_1=authid&term_1=987773&theme=system']
    custom_settings = {
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'Library_catalogue.json'         
    }

    def parse(self, response):
        soup = BeautifulSoup(response.body, 'html.parser')
        records = soup.find(class_='records').find_all('div', class_='record')
        
        author, storage, WD_signature, publisher,  = '', '', '', ''
        pages, year = 0, 0
        for record in records:
            if 'Adres wyd.' in record.text and 'Opis fiz.' in record.text:
                title = record.find(class_='title').text
                title = re.sub(r' [=/].+', '', title)
                try: author = record.find(class_='author').text
                except: pass
                if 'BUW Magazyn' in record.text: storage = 'magazyn'

                infos = record.find_all('tr')
                for info in infos:
                    if 'Klasyfikacja WD' in info.text:
                        WD_signature = info.a.text
                    elif 'Adres wyd.' in info.text:
                        publisher_candidates = info.find_all('span')
                        for publisher in publisher_candidates:
                            if ('class' in publisher and publisher['class'] != 'highlight') or 'class' not in publisher:
                                publisher_with_colon = re.search(r'.+ : ?(.+),?.*(\d{4})', publisher.text)
                                if publisher_with_colon:
                                    publisher = publisher_with_colon
                                else:
                                    publisher = re.search(r'.+? (.+),?.*(\d{4})', publisher.text)
                                    if publisher:
                                        publisher = publisher.group(1)
                                        year = int(publisher.group(2))
                    elif 'Opis fiz.' in info.text:
                        pages = info.find('span').text
                        pages = re.sub(r'\[.+?\]', '', pages)
                        pages = int(re.search(r'\d+', pages).group(0))

                yield {'title': title,
                       'author': author,
                       'storage': storage,
                       'WD_signature': WD_signature,
                       'publisher': publisher,
                       'year': year,
                       'pages': pages}

        pagelinks = soup.find_all(class_='pageLinks')
        for pagelink in pagelinks:
            if 'Następne>' in pagelink.get_text():
                next_page = pagelink.a['href']
                if next_page is not None:
                    yield response.follow(next_page, callback=self.parse)

In [3]:
!scrapy startproject buw
!cd buw

New Scrapy project 'buw', using template directory '/usr/local/lib/python3.6/dist-packages/scrapy/templates/project', created in:
    /content/buw

You can start your first spider with:
    cd buw
    scrapy genspider example example.com


In [4]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

process = CrawlerProcess(get_project_settings())

process.crawl(BookSpider)
process.start()

2020-08-01 15:37:05 [scrapy.utils.log] INFO: Scrapy 2.2.1 started (bot: scrapybot)
2020-08-01 15:37:05 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jul 17 2020, 12:50:27) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.0, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-08-01 15:37:05 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-08-01 15:37:05 [scrapy.crawler] INFO: Overridden settings:
{}
2020-08-01 15:37:05 [scrapy.extensions.telnet] INFO: Telnet Password: f36676c695a6f686
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)
2020-08-01 15:37:05 [scrapy.middleware] INFO: Enabled